In [1]:
## Installation du FrameWork DEAP
!pip3 install deap

In [2]:
#Import d'outils pythons
import random
import operator
import math
import numpy

##Import les outils deap
from deap import base
from deap import creator
from deap import tools
from deap import gp
from deap import algorithms
from math import sqrt

In [3]:
#Création des individus
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [4]:
#Fct qui créer les individus avec des nb aléatoire
#La somme de l'individu est égale a &
def individu(arg):
    ind = numpy.random.dirichlet(numpy.ones(8),size=1)[0]
    return arg(ind)

In [5]:
toolbox = base.Toolbox()
toolbox.register("individual", individu, creator.Individual)

In [6]:
#Fct de fitness avec le rapport de Sharpe
def fitSharpe(ind):
    #Données du tableau (rendement et variance)
    rend1 = [-0.02,-0.01,0.00,0.01,0.02,0.03,0.04,0.05]
    var1 = [0,00.05,0.09,0.01,0.02,0.16,0.20,0.25]
    
    rend = 0
    var = 0
    for i in range(len(ind)):
        rend = rend + ind[i]*rend1[i]
        var = var + ind[i]*var1[i]
    if var > 0 :
        sharpe = rend/sqrt(var)
    else :
        sharpe = 0
    return sharpe,


In [7]:
#Création de la population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", fitSharpe)

In [8]:
##Création de la population initiale sans contraintes
pop = toolbox.population(n=300)

In [9]:
#Evaluation de la population
eval = []*len(pop)
find_max = pop[0]

for ind in pop:    
    ind.fitness.values = fitSharpe(ind)
    if(ind.fitness.values > find_max.fitness.values):
        find_max = ind
print("meilleur individu : ", str(find_max))
print("fitness max : ", str(find_max.fitness.values))

meilleur individu :  [0.006012709438961231, 0.0425220407482908, 0.028790870767138186, 0.021714292919171332, 0.03787263600194607, 0.03838100971918717, 0.7206061086509926, 0.10410033175431266]
fitness max :  (0.08347547084303458,)


In [10]:
def evolution(pop):
    NGEN = 100
    CXPB = 0.5
    MUTPB = 0.2
    for g in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # Apply mutation on the offspring
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring
    return pop

In [11]:
##nouvelle population après 100 généarations avec mutation et crossing over    
pop = evolution(pop)

In [12]:
#Evaluation de la nouvelle population
for ind in pop:    
    ind.fitness.values = fitSharpe(ind)

In [13]:
##Ajout des contraintes de l'énoncé
def cont(ind):   
    bool = False  
    if sum(ind[:3])>=0.1  and sum(ind[:3])>=0 :
        if ind[2] <= 0.1 and ind[2] >= 0:
            if sum(ind[3:5]) <= 0.2 and sum(ind[3:5])>=0 :
                if sum(ind[5:]) <= 0.5 and sum(ind[5:])>= 0:
                    bool = True
                else :
                    bool = False
            else :
                bool = False
        else :
            bool = False
    else :
        bool = False
    return bool

In [14]:
##Création de la population initiale sans contraintes
pop = toolbox.population(n=300)

In [15]:
#Evaluation de la population
eval = []*len(pop)
find_max = pop[0]

for ind in pop:    
    ind.fitness.values = fitSharpe(ind)
    if(ind.fitness.values > find_max.fitness.values):
        find_max = ind
print("meilleur individu : ", str(find_max))
print("fitness max : ", str(find_max.fitness.values))

meilleur individu :  [0.016843194172026416, 0.0009912431585279479, 0.027595708477784968, 0.18828253380358403, 0.0032810807944024063, 0.03320272702720883, 0.03887285379263556, 0.6909306587738299]
fitness max :  (0.08871174266447823,)


In [16]:
##Ajout de la contrainte
toolbox.decorate("evaluate",tools.DeltaPenalty(cont,ind))

In [17]:
##nouvelle population après 100 généarations avec mutation et crossing over    
pop = evolution(pop)

In [18]:
#Evaluation de la nouvelle population
for ind in pop:    
    ind.fitness.values = fitSharpe(ind)

La fitness max n'a pas beaucoup changé entre les deux essaies (avec et sans contrainte)
On constate simplement que nous gardons un ratio proche de 0.08